In [ ]:
##################
#Example of running and plotting of vortex function from PyMeso
##################

import vortex
import numpy as np
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
%matplotlib inline

#run the vortex code with desired properties
x,y,U,V,doppler=vortex.polar_vortex(1,.25,30,[-15,-15],2,20,True,True)

#calculate tangential velocity parameter for plotting
TV= np.sqrt(U**2+V**2)

#plot tangential velocity and doppler returns
fig = plt.figure(figsize=[20,12],facecolor='white')
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
fig.tight_layout()
neg2 = ax1.contourf((x), (y), TV, cmap=cm.Reds,) 
fig.colorbar(neg2, ax=ax1,label='ms$^{-1}$')
neg = ax2.contourf((x), (y), -doppler, cmap=cm.RdBu,) 
fig.colorbar(neg, ax=ax2,label='ms$^{-1}$ (Blue towards radar)')
ax1.set_title('Tangential Velocity',fontsize='24')
ax2.set_title('Doppler Returns',fontsize='24')
ax2.plot(0,0,'ko')
ax1.plot(0,0,'ko')

In [ ]:
##################
#Example of running and plotting of llsd function from PyMeso
##################

import llsd
import pyart
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib.colors as colors

#import radar object
radar = pyart.io.read_mdv("filename")

#call the llsd function form llsd.py
dic=llsd.main(radar,'reflectivity','velocity',0)

#concatenate the selected llsd sweep to an empty array and add the field to the radar object
data2=np.concatenate((dic['data'],np.zeros(((radar.nsweeps-1)*360,dic['data'].shape[1]))),axis=0)
dic2=dic
dic2['data']=data2
radar.add_field('azi_shear', dic2, replace_existing=True)

#define constants as required for plotting
tilt = 0
minlat = -27.7
maxlat = -27.2
minlon = 152.8
maxlon = 153.3
v_min=-.01
v_max=.01
lat_lines = np.arange(minlat,maxlat,.2)
lon_lines = np.arange(minlon, maxlon, .2)

#plot using pyart
display = pyart.graph.RadarMapDisplay(radar)
f = plt.figure(figsize=[9,5],facecolor='white')
plt.rc('font', size=16)
display.plot_ppi_map('azi_shear',tilt,title='Azimuthal Shear', vmin=v_min, vmax=v_max,colorbar_label='s$^{-1}$',
                        lat_lines = lat_lines, lon_lines = lon_lines, 
                        max_lat = maxlat, min_lat =minlat, min_lon = minlon, max_lon = maxlon,
                       cmap=cm.RdBu,resolution = 'h')

ax = plt.gca()